In [17]:
# We use this function to call bx from Python, see below for an example
import subprocess
def call(cmd):
    res = subprocess.check_output([cmd], shell=True)
    return res.strip().decode()

import base58
import hmac
import hashlib

entropy = "openssl rand -hex 24"

## Warmup: Show that the child private key is the parent private key + L256

In [18]:
# use base58.b58decode to decode the xpub or xprv and get the chaincode bytes

parent_xpriv = call(f'{entropy} | bx mnemonic-new | bx mnemonic-to-seed | bx hd-new')
parent_xpub = call(f'bx hd-to-public {parent_xpriv}')
parent_priv = call(f'bx hd-to-ec {parent_xpriv}')
child_xpriv = call(f'bx hd-private --index 0 {parent_xpriv}')
child_xpub = call(f'bx hd-to-public {child_xpriv}')
child_priv = call(f'bx hd-to-ec {child_xpriv}')

xpub_bytes = base58.b58decode(parent_xpub)
chaincode = xpub_bytes[13:13+32]
parent_pub = call(f'bx hd-to-ec {parent_xpub}')
concatenated = parent_pub + "00000000"

l256 = hmac.new(
    chaincode,
    msg=bytes.fromhex(concatenated),
    digestmod=hashlib.sha512
).hexdigest()[:64]

print(call(f'bx ec-add-secrets {parent_priv} {l256}'))
print(child_priv)

310d9b3ca2e13fd6aaa44497e8667a50eaf6f90a13c41b30a7a34415c9ad8e9d
310d9b3ca2e13fd6aaa44497e8667a50eaf6f90a13c41b30a7a34415c9ad8e9d


# Main Exercise: HD Parent Key Exposure

<br>
<img src="images/hd_parent_exposure.jpg" alt="drawing" style="" width="700px"/>



The above figure briefly recaps how parent private keys can be leaked based on child private keys when deriving non-hardened children.

The goal of this exercise is to demonstrate how this works.

If you need some hints, you can check out this implementation in the `pywallet` library:
https://github.com/ranaroussi/pywallet/blob/468622dcf993a27a5b585289b2724986c02a1fbc/pywallet/utils/bip32.py#L380-L429

### Derive new parent key pair

In [20]:
parent_xpriv = call(f'{entropy} | bx mnemonic-new | bx mnemonic-to-seed | bx hd-new')
parent_xpub = call(f'bx hd-to-public {parent_xpriv}')
print(parent_xpriv)
print(parent_xpub)

parent_priv = call(f'bx hd-to-ec {parent_xpriv}')
print(parent_priv)

xprv9s21ZrQH143K2taufeimvGSXoXdfnkoSF1VB3JCpzDkpMRk7e7bChJDYSKMhW1qaJE2Ahaf758wQFoqapV7vMWv8TKLKGGv8DdEKB56hch6
xpub661MyMwAqRbcFNfNmgFnHQPGMZUACDXHcEQmqgcSYZHoEE5GBeuTF6Y2HchuXev9cgpcGPnm9BqmzqPGg235e5VxWh386Lx55kd7jhJqwzo
4f1d99c400c51d0d52f3e59d48cca438d194069b2a5ed3e3e2b5b08a7e2ae5b0


### Derive non-hardenend children

In [26]:
child_xpriv = call(f'bx hd-private --index 0 {parent_xpriv}')
child_priv = call(f'bx hd-to-ec {child_xpriv}')
child_xpub = call(f'bx hd-to-public {child_xpriv}')

print("Exposed material:")
print(f"Parent xpub: {parent_xpub}")
print(f"Child priv: {child_priv}")
print("Child index: 0")

Exposed material:
Parent xpub: xpub661MyMwAqRbcFNfNmgFnHQPGMZUACDXHcEQmqgcSYZHoEE5GBeuTF6Y2HchuXev9cgpcGPnm9BqmzqPGg235e5VxWh386Lx55kd7jhJqwzo
Child priv: 48e95eaf7b7a6b543818c68c77cdc4ca6aadaa85c256ecb88efa27cdafd6db56
Child index: 0


### Step 1: Extract chaincode from parent xpub

In [27]:
xpub_bytes = base58.b58decode(parent_xpub)
chaincode = xpub_bytes[13:13+32]
print(list(chaincode))

[83, 132, 246, 65, 236, 88, 208, 165, 90, 166, 44, 73, 78, 226, 252, 184, 238, 64, 160, 149, 239, 92, 12, 81, 118, 224, 169, 79, 71, 8, 220, 114]


### Step2: Compute L256 bits as HMAC-SHA512(Parent Chain Code, Parent Public Key || Child Index)

In [28]:
parent_pub = call(f'bx hd-to-ec {parent_xpub}')
print(parent_pub)
concatenated = parent_pub + "00000000"

03806174a7e6dcdafddc08ec30b41f21e96f79c0fee8c2348b205c3fc04cf1d26d


In [29]:
l256 = hmac.new(
    chaincode,
    msg=bytes.fromhex(concatenated),
    digestmod=hashlib.sha512
).hexdigest()[:64]

print(l256)

f9cbc4eb7ab54e46e524e0ef2f01209053c880d14740b9106c16d5d001e236e7


### Step 3: Compute Parent private key as child private key - L256

In [35]:
from itertools import takewhile

group_order = "fffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f"

group_order_m_l256 = hex(int(group_order, 16) - int(l256, 16))[2:]
print(group_order_m_l256)

parent_priv_computed = call(f'bx ec-add-secrets {child_priv} {"0"+group_order_m_l256}')
print(parent_priv_computed)
print(parent_priv)
print(f"The two only share the upper {len(list(takewhile(lambda t: t[0]==t[1], zip(parent_priv, parent_priv_computed))))/2} bytes", u"\U0001F914")

6343b14854ab1b91adb1f10d0fedf6fac377f2eb8bf46ef93e92a2efe1dc548
4f1d99c400c51d0d52f3e59d48cca43a16e529b47b1633a822e351fcadf4a09e
4f1d99c400c51d0d52f3e59d48cca438d194069b2a5ed3e3e2b5b08a7e2ae5b0
The two only share the upper 15.5 bytes 🤔
